In [1]:
using DataFrames
using CSV
using LinearAlgebra

In [2]:
# Tu Pham
function predict(linregparam, x)
    predictedResponses = x * linregparam
    return predictedResponses
end

function rsq(predictedResponses, trueResponses, meanPrediction)
    predictiveModelError = 0
    for i in eachindex(trueResponses)
        predictiveModeError += ((trueResponses[i] - predictedResponses[i]) ^ 2)
    end
    baselineModelError = 0
    for i in eachindex(trueResponses)
        baselineModelError += ((trueResponses[i] - meanPrediction) ^ 2)
    end
    r_squared = 1 - (predictiveModelError / baselineModelError)
    return r_squared
end

rsq (generic function with 1 method)